# Parsing and preprocessing LUNA16 Ct images

In [1]:
%%capture
import sys
!{sys.executable} -m pip install SimpleITK==2.2.1 cassandra-driver==3.27.0 diskcache==4.1.0

The `p2ch10/dsets.py` helper module introduces the `Ct` class. In this notebook, I learn
how this class is used to parse and store raw Ct image data for downstream analysis.

In [35]:
import collections
import glob
import numpy as np
import pandas as pd
import SimpleITK as sitk

from pathlib import Path

There are 888 images available in the dataset, stored in 
[DICOM](https://dicom.nema.org/medical/dicom/current/output/chtml/part10/chapter_7.html#:~:text=The%20DICOM%20File%20Format%20provides,contains%20a%20single%20SOP%20Instance.) format that can be parsed with the 
[SimpleITK python module](https://pypi.org/project/SimpleITK/).

Each scan is represented in two files:

1. `.mhd` metadata file
2. `.raw` image file

In [3]:
data_dir = Path("/datasets/luna16")
mhd_paths = list(data_dir.glob('subset*/*.mhd'))
len(mhd_paths)

888

Let's parse and inspect the first scan:

In [4]:
ct_mhd = sitk.ReadImage(mhd_paths[0])
ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

In [5]:
ct_a.shape  # numpy array with shape 121 x 512 x 512

(121, 512, 512)

We can read each image and convert it into a numpy array with the follwowing three spatial dimensions:
    
- `I`: index of the scan => corresponds to the Z-axis in patient coordinates
- `R`: row => corresponds to the Y-axis in patient coordinates
- `C`: column => corresponds ton the X-axis in patient coordinates

(The mapping of IRC to ZYX will be important below.)
    

### Clipping

The intensities reported by the Ct scanner are in [Houdsfield units (HU)](https://radiopaedia.org/articles/hounsfield-unit?lang=us#:~:text=Hounsfield%20units%20(HU)%20are%20a,the%20measured%20attenuation%20coefficients%201.) that roughly track with the type of tissue in the image. 

The values across the dataset range from -3,000 to +2,000 HU.

In [6]:
print(f"mean = {round(float(ct_a.mean()), 2)}, "
      f"min = {ct_a.min()}, "
      f"max = {ct_a.max()}")

mean = -1035.62, min = -3024.0, max = 2103.0


In this project, we only care about nodules in the lung (HU ~ 1), not in bone (HU > 1,000) or outside soft tissues (HU < -1000). We therefore clip values outside the range of interest to limit the impact of outliers.

Numpy's [clip method](https://numpy.org/doc/stable/reference/generated/numpy.clip.html) accepts a range and sets values exceeding the provided limits to the min / max of the range.

In [7]:
ct_a = ct_a.clip(-1000, 1000)
print(f"mean = {round(float(ct_a.mean()), 2)}, "
      f"min = {ct_a.min()}, "
      f"max = {ct_a.max()}")

mean = -603.19, min = -1000.0, max = 1000.0


## Converting patient- to voxel-coordinates

The metadata provided in the `candidates.csv` and `annotations.csv` files includes the position of each nodule in millimeters, e.g. in _patient coordinates_ (X,Y,Z). But the image files use voxel coordinates instead (I,R,C). 

- `X`: right -> left
- `Y`: anterior (front) -> posterior (back)
- `Z`: inferior (feet) -> superior (head)

Luckily, the metadata included in the DICOM files for each scan provides the
information required to translate between the two coordinate systems.

The SimpleITK module provides 
[functionality to read and convert the coordinates](https://simpleitk.readthedocs.io/en/master/fundamentalConcepts.html).
from the `.mhd` file for each scan:

The origin of the patient coordinate system is available via the `.GetOrigin()` method:

In [19]:
origin_xyz = ct_mhd.GetOrigin()
origin_xyz

(-198.100006, -195.0, -335.209991)

The size of the voxel can be retrieved with `.GetSpacing()`:

In [20]:
# voxel size in mm
vxSize_xyz = ct_mhd.GetSpacing()
[round(x, 2) for x in vxSize_xyz]

[0.76, 0.76, 2.5]

It turns out that voxels are _not_ cubes, e.g. the Z-dimension is longer than the 
X and Y dimensions!

The metadata file also includes instructions to rotate or otherwise transform the
I,R,C coordinates to X,Y,Z coordinates, in the form of a 3x3 matrix available with the
`GetDirection()` method. (In this case, we get a diagonal (identity) matrix, e.g. no rotations or other transformations are necessary.)

In [29]:
direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)
direction_a

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

To move from I,R,C to X,Y,Z, we need to execute the following operations:

1. Flip the coordinates from IRC to CRI, to align with XYZ .
2. Scale the indices with the voxel sizes.
3. Matrix-multiply with the directions matrix, using @ in Python.
4. Add the offset for the origin.

As a first step, need to reorder the dimensions from I-R-C to C-R-I, 
to match the dimensions of the X-Y-Z coordinate system.

In [ ]:
# example illustrating the effect of [::-1] slicing
np.array(range(9)).reshape(3,3)[::-1]

The following two helper functions in `utils/util.py` convert
between the two coordinate systems:

In [18]:
def irc2xyz(coord_irc, origin_xyz, vxSize_xyz, direction_a):
    cri_a = np.array(coord_irc)[::-1]  # swap dims
    origin_a = np.array(origin_xyz)  # convert to numpy array
    vxSize_a = np.array(vxSize_xyz)  # convert to numpy array
    coords_xyz = (direction_a @ (cri_a * vxSize_a)) + origin_a
    return XyzTuple(*coords_xyz)

def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))


## The Ct class

The `Ct` class is the main container for the images. It is initalized with a
`series_uid` and loads the corresponding images from disk. It is implemented in the `p2ch10/dsets.py` module.

It's `getRawCandidate()` method returns a cropped section with a user-specified `width_irc` from the (large) Ct image, centered on a candidate nodule (`center_xyz`).

In [42]:
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])

class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob(
            '/datasets/luna16/subset*/{}.mhd'.format(series_uid)
        )[0]

        ct_mhd = sitk.ReadImage(mhd_path)
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
        ct_a.clip(-1000, 1000, ct_a)

        self.series_uid = series_uid
        self.hu_a = ct_a

        self.origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
        self.vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
        self.direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    def getRawCandidate(self, center_xyz, width_irc):
        center_irc = xyz2irc(
            center_xyz,
            self.origin_xyz,
            self.vxSize_xyz,
            self.direction_a,
        )

        slice_list = []
        for axis, center_val in enumerate(center_irc):
            # define start and of the region for the current axis
            start_ndx = int(round(center_val - width_irc[axis]/2))
            end_ndx = int(start_ndx + width_irc[axis])

            assert center_val >= 0 and center_val < self.hu_a.shape[axis], \
              repr([self.series_uid, center_xyz, self.origin_xyz, self.vxSize_xyz, center_irc, axis])

            if start_ndx < 0:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                start_ndx = 0
                end_ndx = int(width_irc[axis])

            if end_ndx > self.hu_a.shape[axis]:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                end_ndx = self.hu_a.shape[axis]
                start_ndx = int(self.hu_a.shape[axis] - width_irc[axis])

            slice_list.append(slice(start_ndx, end_ndx))

        ct_chunk = self.hu_a[tuple(slice_list)]

        return ct_chunk, center_irc


In [43]:
uid = mhd_paths[0].stem
ct = Ct(uid)